TARGET LABEL: Z_Score_HelpfulVotes
SCALER: STANDARD SCALER
ANN'S : 
    Combination 1: 7-100-100-1
    
    Combination 2:  7-4-4-1

    Combination 3:  7-50-100-200-100-50-20-1

    Combination 4: 7-50-50-11

    Combination 5: 7-20-20-11
    
Here we will use the absolute value of (Sentiment - Mean of Sentiments)

In [1]:
import numpy as np
import pandas as pd
import keras
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


In [2]:
dataset = pd.read_csv(r"/Users/t_velpac/mission/WorkingCopy/final_csv_files/Abs_Sentiment-Mean.csv")
dataset = dataset.drop(columns=['Date', 'Helpful_Votes', 'Z_Score_Words', 'Z_Score_Paragraphs', 'Sentiment_Polarity'])
cols = (list(dataset.columns))
cols.remove('Z_Score_HelpfulVotes')
cols.append('Z_Score_HelpfulVotes')
dataset = dataset[cols]
dataset.head()

,Stars,Words,Paragraphs,No.break tags,Percentage_Upper_Case,Percentage_Lower_Case,Avg_len_paragraph_per_review,Absolute(Sentiment-Mean),Z_Score_HelpfulVotes
0,4,268,8,14,3,90,166.125000,0.118601,6.515421
1,1,130,2,2,2,94,352.500000,0.140716,3.557805
2,1,110,1,0,3,90,556.000000,0.173722,3.557805
3,5,110,3,4,3,92,188.666667,0.125016,70.014530
4,1,39,1,0,3,96,216.000000,0.431510,2.347872


In [3]:
""" Splitting the independent variables from the dependent variable """
X = dataset.iloc[:,0:-1].values
y = dataset.iloc[:,-1].values

In [4]:
"""Splitting the data into training data and testing data"""
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size=0.2,random_state=0)

"""Scaling the data using StandardScaler from sklearn package"""
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)

In [5]:
""" Building a 3 layer ANN with high number of nodes """

regressor = Sequential()
regressor.add(Dense(100, kernel_initializer = 'normal',activation = 'relu',input_dim = 8))
regressor.add(Dense(100, kernel_initializer = 'normal', activation = 'relu'))
regressor.add(Dense(1, kernel_initializer = 'normal'))

""" Compiling the regressor """
regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')

""" Fitting the Artifilial Neural Network to our training data """
regressor.fit(X_train, y_train, batch_size=5, epochs=100, verbose = 1)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch 1/100
156392/156392 [==============================] - 34s 217us/step - loss: 1.0328
Epoch 2/100
156392/156392 [==============================] - 33s 211us/step - loss: 1.0305 - ETA: 0s - loss: 1.
Epoch 3/100
156392/156392 [==============================] - 33s 214us/step - loss: 1.0304
Epoch 4/100
156392/156392 [==============================] - 34s 219us/step - loss: 1.0299
Epoch 5/100
156392/156392 [==============================] - 33s 211us/step - loss: 1.0262
Epoch 6/100
156392/156392 [==============================] - 33s 210us/step - loss: 1.0251
Epoch 7/100
156392/156392 [==============================] - 33s 211us/step - loss: 1.0273
Epoch 8/100
156392/156392 [==============================] - 34s 215us/step - loss: 1.0264 
Epoch 9/100
156392/156392 [==============================] - 33s 212us/step - loss: 1.02700s - loss: 1
Epoch 10/100
156392/156392 

156392/156392 [==============================] - 34s 215us/step - loss: 1.0291
Epoch 78/100
156392/156392 [==============================] - 33s 212us/step - loss: 1.0325
Epoch 79/100
156392/156392 [==============================] - 34s 217us/step - loss: 1.0291
Epoch 80/100
156392/156392 [==============================] - 33s 211us/step - loss: 1.02950s - loss: 1.02 - ETA: 0s - los
Epoch 81/100
156392/156392 [==============================] - 34s 215us/step - loss: 1.0296
Epoch 82/100
156392/156392 [==============================] - 34s 218us/step - loss: 1.03090s - loss: 1
Epoch 83/100
156392/156392 [==============================] - 35s 222us/step - loss: 1.0297
Epoch 84/100
156392/156392 [==============================] - 34s 217us/step - loss: 1.0291
Epoch 85/100
156392/156392 [==============================] - 35s 225us/step - loss: 1.0301
Epoch 86/100
156392/156392 [==============================] - 33s 212us/step - loss: 1.0301
Epoch 87/100
156392/156392 [======================

In [6]:
from sklearn.metrics import r2_score

In [7]:
""" Predicting the values for Helpful Votes on the test data """
y_pred = regressor.predict(X_test)

""" Creating a dataframe to compare the actual values against predicted values """
y_pred = y_pred.reshape(39099,)
temp = {'Actual Values': y_test,'Predicted Values': y_pred}
y_compare = pd.DataFrame(temp)

""" Calculating the Mean Squared Error to estimate the efficiency of the ANN"""
# We are calculating this MSE in two steps. Don't get confused.
y_compare['Mean Squared Error'] = (np.diff(y_compare.values) ** 2)
y_compare['Mean Squared Error'] = np.mean(y_compare['Mean Squared Error'])

# Now calculating the Root Mean Squared Error(RMSE)
y_compare['Root Mean Squared Error'] = y_compare['Mean Squared Error']**0.5

# Calculating Mean Absolute Error
y_compare['Mean Abs. Error'] = np.mean(abs(y_compare['Actual Values'] - y_compare['Predicted Values']))

# Calculating R-Squared value
y_compare['R_Squared'] = r2_score(y_test, y_pred)

y_compare

,Actual Values,Predicted Values,Mean Squared Error,Root Mean Squared Error,Mean Abs. Error,R_Squared
0,-0.027183,0.085131,0.797038,0.89277,0.106397,0.018846
1,-0.032077,-0.064950,0.797038,0.89277,0.106397,0.018846
2,-0.031418,-0.047652,0.797038,0.89277,0.106397,0.018846
3,-0.051497,-0.048188,0.797038,0.89277,0.106397,0.018846
4,-0.029573,0.006704,0.797038,0.89277,0.106397,0.018846
5,-0.061395,-0.062344,0.797038,0.89277,0.106397,0.018846
6,-0.026445,0.150848,0.797038,0.89277,0.106397,0.018846
7,0.497499,0.002593,0.797038,0.89277,0.106397,0.018846
8,-0.090490,-0.072232,0.797038,0.89277,0.106397,0.018846
9,-0.031418,-0.015826,0.797038,0.89277,0.106397,0.018846


In [8]:
""" Now we will also predict the y values on the training set just to calculate MSE and RMSE """

y_pred_train = regressor.predict(X_train)

""" Creating a dataframe to compare the actual values against the predicted values for the training set """
y_pred_train = y_pred_train.reshape(156392,)
temp_train = {'Actual Values(Training)':y_train, 'Predicted Values(Training)': y_pred_train }
y_compare_train = pd.DataFrame(temp_train)

""" Calculating the Mean Squared Error to estimate the efficiency of the ANN on TRAINING SET"""
# We are calculating this MSE in two steps. Don't get confused.
y_compare_train['Mean Squared Error'] = (np.diff(y_compare_train.values) ** 2)
y_compare_train['Mean Squared Error'] = np.mean(y_compare_train['Mean Squared Error'])

# Now calculating the Root Mean Squared Error(RMSE)
y_compare_train['Root Mean Squared Error'] = y_compare_train['Mean Squared Error']**0.5

# Calculating Mean Absolute Error
y_compare_train['Mean Abs. Error'] = np.mean(abs(y_compare_train['Actual Values(Training)'] - y_compare_train['Predicted Values(Training)']))

# Calculating R-Squared value
y_compare_train['R_Squared'] = r2_score(y_train, y_pred_train)

y_compare_train

,Actual Values(Training),Predicted Values(Training),Mean Squared Error,Root Mean Squared Error,Mean Abs. Error,R_Squared
0,-0.084480,-0.024107,1.022265,1.011071,0.112606,0.023537
1,-0.072782,0.903971,1.022265,1.011071,0.112606,0.023537
2,-0.112014,-0.048100,1.022265,1.011071,0.112606,0.023537
3,-0.059294,-0.042276,1.022265,1.011071,0.112606,0.023537
4,-0.046479,-0.052191,1.022265,1.011071,0.112606,0.023537
5,0.009217,-0.032304,1.022265,1.011071,0.112606,0.023537
6,-0.008194,-0.013043,1.022265,1.011071,0.112606,0.023537
7,-0.027183,-0.046464,1.022265,1.011071,0.112606,0.023537
8,-0.033595,-0.058181,1.022265,1.011071,0.112606,0.023537
9,-0.090653,-0.012801,1.022265,1.011071,0.112606,0.023537


In [ ]:
""" Building a 3 layer ANN with 2 layers of 4 nodes """

regressor_1 = Sequential()
regressor_1.add(Dense(4, kernel_initializer = 'normal',activation = 'relu',input_dim = 8))
regressor_1.add(Dense(4, kernel_initializer = 'normal', activation = 'relu'))
regressor_1.add(Dense(1, kernel_initializer = 'normal'))

""" Compiling the regressor """
regressor_1.compile(optimizer = 'adam', loss = 'mean_squared_error')

""" Fitting the Artifilial Neural Network to our training data """
regressor_1.fit(X_train, y_train, batch_size=5, epochs=100, verbose = 1)

Epoch 1/100
156392/156392 [==============================] - 36s 230us/step - loss: 1.0263
Epoch 2/100
156392/156392 [==============================] - 35s 221us/step - loss: 1.02460s - loss: 1. - ETA: 0s - 
Epoch 3/100
156392/156392 [==============================] - 35s 224us/step - loss: 1.0248
Epoch 4/100
156392/156392 [==============================] - 34s 218us/step - loss: 1.0236
Epoch 5/100
156392/156392 [==============================] - 35s 224us/step - loss: 1.0237
Epoch 6/100
156392/156392 [==============================] - 34s 218us/step - loss: 1.0229
Epoch 7/100
156392/156392 [==============================] - 34s 218us/step - loss: 1.0262
Epoch 8/100
156392/156392 [==============================] - 34s 216us/step - loss: 1.0240
Epoch 9/100
156392/156392 [==============================] - 34s 216us/step - loss: 1.0255
Epoch 10/100
156392/156392 [==============================] - 34s 216us/step - loss: 1.0241 ETA: 
Epoch 11/100
156392/156392 [=============================

156392/156392 [==============================] - 34s 220us/step - loss: 1.02222s - loss: 1.0 - ETA: 2s - loss:  
Epoch 87/100
156392/156392 [==============================] - 35s 227us/step - loss: 1.0231
Epoch 88/100
156392/156392 [==============================] - 36s 231us/step - loss: 1.0238
Epoch 89/100
156392/156392 [==============================] - 36s 232us/step - loss: 1.0235A: 9s - lo - ETA: 9s - l - ETA: 8s - loss: 1 - ETA: 8s - loss: 0.994 - ETA: 8s - loss: 0.99 - ETA: 8s - l
Epoch 90/100
156392/156392 [==============================] - 35s 224us/step - loss: 1.02228s  - ETA: 8s - E - ET - ETA: 4s - loss: 1.06 - ETA: 4s - lo - ETA: 
Epoch 91/100
156392/156392 [==============================] - 36s 229us/step - loss: 1.0240A: 7s - loss: 1.09 - ET
Epoch 92/100
156392/156392 [==============================] - 36s 229us/step - loss: 1.0228
Epoch 93/100
156392/156392 [==============================] - 35s 224us/step - loss: 1.0232
Epoch 94/100
156392/156392 [===================

In [ ]:
""" Predicting the values for Helpful Votes on the test data """
y_pred_1 = regressor_1.predict(X_test)

""" Creating a dataframe to compare the actual values against predicted values """
y_pred_1 = y_pred_1.reshape(39099,)
temp = {'Actual Values': y_test,'Predicted Values': y_pred_1}
y_compare_1 = pd.DataFrame(temp)

""" Calculating the Mean Squared Error to estimate the efficiency of the ANN"""
# We are calculating this MSE in two steps. Don't get confused.
y_compare_1['Mean Squared Error'] = (np.diff(y_compare_1.values) ** 2)
y_compare_1['Mean Squared Error'] = np.mean(y_compare_1['Mean Squared Error'])

# Now calculating the Root Mean Squared Error(RMSE)
y_compare_1['Root Mean Squared Error'] = y_compare_1['Mean Squared Error']**0.5

# Calculating Mean Absolute Error
y_compare_1['Mean Abs. Error'] = np.mean(abs(y_compare_1['Actual Values'] - y_compare_1['Predicted Values']))

# Calculating R-Squared value
y_compare_1['R_Squared'] = r2_score(y_test, y_pred_1)

y_compare_1

,Actual Values,Predicted Values,Mean Squared Error,Root Mean Squared Error,Mean Abs. Error,R_Squared
0,-0.027183,0.057887,0.796335,0.892376,0.105453,0.019711
1,-0.032077,-0.053901,0.796335,0.892376,0.105453,0.019711
2,-0.031418,-0.053901,0.796335,0.892376,0.105453,0.019711
3,-0.051497,-0.053901,0.796335,0.892376,0.105453,0.019711
4,-0.029573,-0.043771,0.796335,0.892376,0.105453,0.019711
5,-0.061395,-0.050927,0.796335,0.892376,0.105453,0.019711
6,-0.026445,0.122591,0.796335,0.892376,0.105453,0.019711
7,0.497499,-0.009599,0.796335,0.892376,0.105453,0.019711
8,-0.090490,-0.053901,0.796335,0.892376,0.105453,0.019711
9,-0.031418,-0.040142,0.796335,0.892376,0.105453,0.019711


In [ ]:
""" Now we will also predict the y values on the training set just to calculate MSE and RMSE """

y_pred_train_1 = regressor_1.predict(X_train)

""" Creating a dataframe to compare the actual values against the predicted values for the training set """
y_pred_train_1 = y_pred_train_1.reshape(156392,)
temp_train = {'Actual Values(Training)':y_train, 'Predicted Values(Training)': y_pred_train_1 }
y_compare_train_1 = pd.DataFrame(temp_train)

""" Calculating the Mean Squared Error to estimate the efficiency of the ANN on TRAINING SET"""
# We are calculating this MSE in two steps. Don't get confused.
y_compare_train_1['Mean Squared Error'] = (np.diff(y_compare_train_1.values) ** 2)
y_compare_train_1['Mean Squared Error'] = np.mean(y_compare_train_1['Mean Squared Error'])

# Now calculating the Root Mean Squared Error(RMSE)
y_compare_train_1['Root Mean Squared Error'] = y_compare_train_1['Mean Squared Error']**0.5

# Calculating Mean Absolute Error
y_compare_train_1['Mean Abs. Error'] = np.mean(abs(y_compare_train_1['Actual Values(Training)'] - y_compare_train_1['Predicted Values(Training)']))


# Calculating R-Squared value
y_compare_train_1['R_Squared'] = r2_score(y_train, y_pred_train_1)

y_compare_train_1

,Actual Values(Training),Predicted Values(Training),Mean Squared Error,Root Mean Squared Error,Mean Abs. Error,R_Squared
0,-0.084480,-0.049161,1.024425,1.012139,0.111477,0.021474
1,-0.072782,0.587157,1.024425,1.012139,0.111477,0.021474
2,-0.112014,-0.053901,1.024425,1.012139,0.111477,0.021474
3,-0.059294,-0.046305,1.024425,1.012139,0.111477,0.021474
4,-0.046479,-0.049931,1.024425,1.012139,0.111477,0.021474
5,0.009217,-0.038858,1.024425,1.012139,0.111477,0.021474
6,-0.008194,-0.048977,1.024425,1.012139,0.111477,0.021474
7,-0.027183,-0.053901,1.024425,1.012139,0.111477,0.021474
8,-0.033595,-0.053901,1.024425,1.012139,0.111477,0.021474
9,-0.090653,-0.042259,1.024425,1.012139,0.111477,0.021474


In [ ]:
"""


Now, we will train an ANN with a very high number of nodes. And comparitively more layers.

8-50-100-200-100-50-20-1
"""

'\n\n\nNow, we will train an ANN with a very high number of nodes. And comparitively more layers.\n\n8-50-100-200-100-50-20-1\n'

In [ ]:
""" Building a deep and wide ANN with high number of nodes """

from keras.layers.advanced_activations import LeakyReLU
regressor_2 = Sequential()
regressor_2.add(Dense(50, kernel_initializer = 'normal',input_dim = 8))
regressor_2.add(LeakyReLU(alpha=0.05))
regressor_2.add(Dense(100, kernel_initializer = 'normal'))
regressor_2.add(LeakyReLU(alpha=0.05))
regressor_2.add(Dense(200, kernel_initializer = 'normal'))
regressor_2.add(LeakyReLU(alpha=0.05))
regressor_2.add(Dense(100, kernel_initializer = 'normal'))
regressor_2.add(LeakyReLU(alpha=0.05))
regressor_2.add(Dense(50, kernel_initializer = 'normal'))
regressor_2.add(LeakyReLU(alpha=0.05))
regressor_2.add(Dense(20, kernel_initializer = 'normal'))
regressor_2.add(LeakyReLU(alpha=0.05))
regressor_2.add(Dense(1, kernel_initializer = 'normal'))

""" Compiling the regressor """
regressor_2.compile(optimizer = 'adam', loss = 'mean_squared_error')

""" Fitting the Artifilial Neural Network to our training data """
regressor_2.fit(X_train, y_train, batch_size=5, epochs=100, verbose = 1)

Epoch 1/100
156392/156392 [==============================] - 56s 360us/step - loss: 1.0353
Epoch 2/100
156392/156392 [==============================] - 55s 355us/step - loss: 1.03274s - loss: 1.029 - ETA: 4s - loss: 1 - - ETA: 1s - loss: 1 - ETA: 1s - l - ETA: 0s 
Epoch 3/100
156392/156392 [==============================] - 55s 354us/step - loss: 1.0327
Epoch 4/100
156392/156392 [==============================] - 56s 356us/step - loss: 1.0325
Epoch 5/100
156392/156392 [==============================] - 55s 352us/step - loss: 1.0316
Epoch 6/100
156392/156392 [==============================] - 55s 353us/step - loss: 1.0316
Epoch 7/100
156392/156392 [==============================] - 56s 356us/step - loss: 1.03350s - loss: - ETA: 0s - los
Epoch 8/100
156392/156392 [==============================] - 57s 362us/step - loss: 1.03231s - - 
Epoch 9/100
156392/156392 [==============================] - 56s 359us/step - loss: 1.0334
Epoch 10/100
156392/156392 [==============================] - 56s

156392/156392 [==============================] - 54s 345us/step - loss: 1.0412
Epoch 82/100
156392/156392 [==============================] - 54s 346us/step - loss: 1.0336
Epoch 83/100
156392/156392 [==============================] - 54s 343us/step - loss: 1.03752
Epoch 84/100
156392/156392 [==============================] - 55s 349us/step - loss: 109.46151s - l - ETA: 0s  - ETA: 0s - loss: 109.490
Epoch 85/100
156392/156392 [==============================] - 54s 345us/step - loss: 1.0367
Epoch 86/100
156392/156392 [==============================] - 54s 348us/step - loss: 1.1149
Epoch 87/100
156392/156392 [==============================] - 54s 346us/step - loss: 5013.2901s - loss: 5 - - E - ETA: 0s - loss: 5
Epoch 88/100
156392/156392 [==============================] - 55s 350us/step - loss: 1572.6310
Epoch 89/100
156392/156392 [==============================] - 55s 349us/step - loss: 1.8803
Epoch 90/100
156392/156392 [==============================] - 54s 347us/step - loss: 1.8423
Epoc

In [ ]:
""" Predicting the values for Helpful Votes on the test data """
y_pred_2 = regressor_2.predict(X_test)

""" Creating a dataframe to compare the actual values against predicted values """
y_pred_2 = y_pred_2.reshape(39099,)
temp = {'Actual Values': y_test,'Predicted Values': y_pred_2}
y_compare_2 = pd.DataFrame(temp)

""" Calculating the Mean Squared Error to estimate the efficiency of the ANN"""
# We are calculating this MSE in two steps. Don't get confused.
y_compare_2['Mean Squared Error'] = (np.diff(y_compare_2.values) ** 2)
y_compare_2['Mean Squared Error'] = np.mean(y_compare_2['Mean Squared Error'])

# Now calculating the Root Mean Squared Error(RMSE)
y_compare_2['Root Mean Squared Error'] = y_compare_2['Mean Squared Error']**0.5

# Calculating Mean Absolute Error
y_compare_2['Mean Abs. Error'] = np.mean(abs(y_compare_2['Actual Values'] - y_compare_2['Predicted Values']))

# Calculating R-Squared value
y_compare_2['R_Squared'] = r2_score(y_test, y_pred_2)

y_compare_2

,Actual Values,Predicted Values,Mean Squared Error,Root Mean Squared Error,Mean Abs. Error,R_Squared
0,-0.027183,-0.002557,0.801785,0.895425,0.098569,0.013002
1,-0.032077,-0.039979,0.801785,0.895425,0.098569,0.013002
2,-0.031418,-0.046875,0.801785,0.895425,0.098569,0.013002
3,-0.051497,-0.041220,0.801785,0.895425,0.098569,0.013002
4,-0.029573,-0.005757,0.801785,0.895425,0.098569,0.013002
5,-0.061395,-0.042341,0.801785,0.895425,0.098569,0.013002
6,-0.026445,0.150429,0.801785,0.895425,0.098569,0.013002
7,0.497499,-0.011793,0.801785,0.895425,0.098569,0.013002
8,-0.090490,-0.037701,0.801785,0.895425,0.098569,0.013002
9,-0.031418,0.021603,0.801785,0.895425,0.098569,0.013002


In [ ]:
""" Now we will also predict the y values on the training set just to calculate MSE and RMSE """

y_pred_train_2 = regressor_2.predict(X_train)

""" Creating a dataframe to compare the actual values against the predicted values for the training set """
y_pred_train_2 = y_pred_train_2.reshape(156392,)
temp_train = {'Actual Values(Training)':y_train, 'Predicted Values(Training)': y_pred_train_2 }
y_compare_train_2 = pd.DataFrame(temp_train)

""" Calculating the Mean Squared Error to estimate the efficiency of the ANN on TRAINING SET"""
# We are calculating this MSE in two steps. Don't get confused.
y_compare_train_2['Mean Squared Error'] = (np.diff(y_compare_train_2.values) ** 2)
y_compare_train_2['Mean Squared Error'] = np.mean(y_compare_train_2['Mean Squared Error'])

# Now calculating the Root Mean Squared Error(RMSE)
y_compare_train_2['Root Mean Squared Error'] = y_compare_train_2['Mean Squared Error']**0.5

# Calculating Mean Absolute Error
y_compare_train_2['Mean Abs. Error'] = np.mean(abs(y_compare_train_2['Actual Values(Training)'] - y_compare_train_2['Predicted Values(Training)']))

# Calculating R-Squared value
y_compare_train['R_Squared'] = r2_score(y_train, y_pred_train_2)

y_compare_train_2

,Actual Values(Training),Predicted Values(Training),Mean Squared Error,Root Mean Squared Error,Mean Abs. Error
0,-0.084480,-0.033485,1.035409,1.017551,0.104573
1,-0.072782,0.263526,1.035409,1.017551,0.104573
2,-0.112014,-0.022144,1.035409,1.017551,0.104573
3,-0.059294,-0.038433,1.035409,1.017551,0.104573
4,-0.046479,-0.042170,1.035409,1.017551,0.104573
5,0.009217,-0.020358,1.035409,1.017551,0.104573
6,-0.008194,0.000324,1.035409,1.017551,0.104573
7,-0.027183,-0.045689,1.035409,1.017551,0.104573
8,-0.033595,-0.042042,1.035409,1.017551,0.104573
9,-0.090653,-0.005012,1.035409,1.017551,0.104573


In [ ]:
""" Building a 3 layer ANN with 2 layers of 50 nodes """

regressor_3 = Sequential()
regressor_3.add(Dense(50, kernel_initializer = 'normal',activation = 'relu',input_dim = 8))
regressor_3.add(Dense(50, kernel_initializer = 'normal', activation = 'relu'))
regressor_3.add(Dense(1, kernel_initializer = 'normal'))

""" Compiling the regressor """
regressor_3.compile(optimizer = 'adam', loss = 'mean_squared_error')

""" Fitting the Artifilial Neural Network to our training data """
regressor_3.fit(X_train, y_train, batch_size=5, epochs=100, verbose = 1)

Epoch 1/100
156392/156392 [==============================] - 38s 241us/step - loss: 1.0324
Epoch 2/100
156392/156392 [==============================] - 36s 231us/step - loss: 1.02775s -  - ETA: 4s -  - ETA: 0s - loss: 1
Epoch 3/100
156392/156392 [==============================] - 51s 323us/step - loss: 1.02966s - ETA: - ETA: 1s - loss:  - E
Epoch 4/100
156392/156392 [==============================] - 37s 239us/step - loss: 1.0279TA: 8s - loss: 0.9 - E - ETA: 2s - loss: 1.02 - ETA: 1s - loss: 1. -  - ETA: 0s - l
Epoch 5/100
156392/156392 [==============================] - 35s 227us/step - loss: 1.0275
Epoch 6/100
156392/156392 [==============================] - 36s 227us/step - loss: 1.0271
Epoch 7/100
156392/156392 [==============================] - 37s 240us/step - loss: 1.02784 - ETA: 1s - loss: 0 - ETA: 
Epoch 8/100
156392/156392 [==============================] - 39s 250us/step - loss: 1.02793s - loss: 
Epoch 9/100
156392/156392 [==============================] - 40s 256us/step - l

In [ ]:
""" Predicting the values for Helpful Votes on the test data """
y_pred_3 = regressor_3.predict(X_test)

""" Creating a dataframe to compare the actual values against predicted values """
y_pred_3 = y_pred_3.reshape(39099,)
temp = {'Actual Values': y_test,'Predicted Values': y_pred_3}
y_compare_3 = pd.DataFrame(temp)

""" Calculating the Mean Squared Error to estimate the efficiency of the ANN"""
# We are calculating this MSE in two steps. Don't get confused.
y_compare_3['Mean Squared Error'] = (np.diff(y_compare_3.values) ** 2)
y_compare_3['Mean Squared Error'] = np.mean(y_compare_3['Mean Squared Error'])

# Now calculating the Root Mean Squared Error(RMSE)
y_compare_3['Root Mean Squared Error'] = y_compare_3['Mean Squared Error']**0.5

# Calculating Mean Absolute Error
y_compare_3['Mean Abs. Error'] = np.mean(abs(y_compare_3['Actual Values'] - y_compare_3['Predicted Values']))

# Calculating R-Squared value
y_compare_3['R_Squared'] = r2_score(y_test, y_pred_3)

y_compare_3

In [ ]:
""" Now we will also predict the y values on the training set just to calculate MSE and RMSE """

y_pred_train_3 = regressor_3.predict(X_train)

""" Creating a dataframe to compare the actual values against the predicted values for the training set """
y_pred_train_3 = y_pred_train_3.reshape(156392,)
temp_train = {'Actual Values(Training)':y_train, 'Predicted Values(Training)': y_pred_train_3 }
y_compare_train_3 = pd.DataFrame(temp_train)

""" Calculating the Mean Squared Error to estimate the efficiency of the ANN on TRAINING SET"""
# We are calculating this MSE in two steps. Don't get confused.
y_compare_train_3['Mean Squared Error'] = (np.diff(y_compare_train_3.values) ** 2)
y_compare_train_3['Mean Squared Error'] = np.mean(y_compare_train_3['Mean Squared Error'])

# Now calculating the Root Mean Squared Error(RMSE)
y_compare_train_3['Root Mean Squared Error'] = y_compare_train_3['Mean Squared Error']**0.5

# Calculating Mean Absolute Error
y_compare_train_3['Mean Abs. Error'] = np.mean(abs(y_compare_train_3['Actual Values(Training)'] - y_compare_train_3['Predicted Values(Training)']))


# Calculating R-Squared value
y_compare_train_3['R_Squared'] = r2_score(y_train, y_pred_train_3)

y_compare_train_3

In [ ]:
""" Building a 3 layer ANN with 2 layers of 20 nodes """

regressor_4 = Sequential()
regressor_4.add(Dense(20, kernel_initializer = 'normal',activation = 'relu',input_dim = 8))
regressor_4.add(Dense(20, kernel_initializer = 'normal', activation = 'relu'))
regressor_4.add(Dense(1, kernel_initializer = 'normal'))

""" Compiling the regressor """
regressor_4.compile(optimizer = 'adam', loss = 'mean_squared_error')

""" Fitting the Artifilial Neural Network to our training data """
regressor_4.fit(X_train, y_train, batch_size=5, epochs=100, verbose = 1)

In [ ]:
""" Predicting the values for Helpful Votes on the test data """
y_pred_4 = regressor_4.predict(X_test)

""" Creating a dataframe to compare the actual values against predicted values """
y_pred_4 = y_pred_4.reshape(39099,)
temp = {'Actual Values': y_test,'Predicted Values': y_pred_4}
y_compare_4 = pd.DataFrame(temp)

""" Calculating the Mean Squared Error to estimate the efficiency of the ANN"""
# We are calculating this MSE in two steps. Don't get confused.
y_compare_4['Mean Squared Error'] = (np.diff(y_compare_4.values) ** 2)
y_compare_4['Mean Squared Error'] = np.mean(y_compare_4['Mean Squared Error'])

# Now calculating the Root Mean Squared Error(RMSE)
y_compare_4['Root Mean Squared Error'] = y_compare_4['Mean Squared Error']**0.5

# Calculating Mean Absolute Error
y_compare_4['Mean Abs. Error'] = np.mean(abs(y_compare_4['Actual Values'] - y_compare_4['Predicted Values']))

# Calculating R-Squared value
y_compare_4['R_Squared'] = r2_score(y_test, y_pred_4)

y_compare_4

In [ ]:
""" Now we will also predict the y values on the training set just to calculate MSE and RMSE """

y_pred_train_4 = regressor_4.predict(X_train)

""" Creating a dataframe to compare the actual values against the predicted values for the training set """
y_pred_train_4 = y_pred_train_4.reshape(156392,)
temp_train = {'Actual Values(Training)':y_train, 'Predicted Values(Training)': y_pred_train_4 }
y_compare_train_4 = pd.DataFrame(temp_train)

""" Calculating the Mean Squared Error to estimate the efficiency of the ANN on TRAINING SET"""
# We are calculating this MSE in two steps. Don't get confused.
y_compare_train_4['Mean Squared Error'] = (np.diff(y_compare_train_4.values) ** 2)
y_compare_train_4['Mean Squared Error'] = np.mean(y_compare_train_4['Mean Squared Error'])

# Now calculating the Root Mean Squared Error(RMSE)
y_compare_train_4['Root Mean Squared Error'] = y_compare_train_4['Mean Squared Error']**0.5

# Calculating Mean Absolute Error
y_compare_train_4['Mean Abs. Error'] = np.mean(abs(y_compare_train_4['Actual Values(Training)'] - y_compare_train_4['Predicted Values(Training)']))

# Calculating R-Squared value
y_compare_train_4['R_Squared'] = r2_score(y_train, y_pred_train_4)

y_compare_train_4